In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
from PIL import Image

In [16]:
data_path = "dataset/categories/cleaning/"

In [ ]:

transform_train = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

transform_val = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [ ]:

dataset = datasets.ImageFolder(data_path, transform=transform_train)
num_classes = len(dataset.classes)
print("Classes:", dataset.classes)


Classes: ['high', 'low', 'medium']


In [ ]:

total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size   = int(0.15 * total_size)
test_size  = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_dataset.dataset.transform = transform_train
val_dataset.dataset.transform   = transform_val
test_dataset.dataset.transform  = transform_val

In [ ]:

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )

        self.pool = nn.MaxPool2d(2)

        self.flatten = nn.Flatten()

        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.pool(self.conv1(x))
        x = self.pool(self.conv2(x))
        x = self.pool(self.conv3(x))
        x = self.flatten(x)
        x = self.fc_layers(x)
        return x


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=3)

In [ ]:
def evaluate(loader):
    model.eval()
    correct, total = 0, 0
    running_loss = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return running_loss / len(loader), 100 * correct / total

In [ ]:
epochs = 20
patience = 5
best_val_loss = float('inf')
counter = 0

for epoch in range(epochs):
    model.train()
    train_loss = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    val_loss, val_acc = evaluate(val_loader)
    scheduler.step(val_loss)

    print(f"\nEpoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss/len(train_loader):.4f}")
    print(f"Val Loss:   {val_loss:.4f}")
    print(f"Val Acc:    {val_acc:.2f}%")
    print("--------------------------------")

  
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), "best_model.pth")
    else:
        counter += 1
        if counter >= patience:
            print(f"No improvement for {patience} epochs. Early stopping.")
            break

Epoch 1/20: 100%|██████████| 14/14 [00:07<00:00,  1.83it/s]



Epoch 1/20
Train Loss: 8.8105
Val Loss:   11.5687
Val Acc:    31.11%
--------------------------------


Epoch 2/20: 100%|██████████| 14/14 [00:07<00:00,  1.91it/s]



Epoch 2/20
Train Loss: 2.6567
Val Loss:   1.8458
Val Acc:    38.89%
--------------------------------


Epoch 3/20: 100%|██████████| 14/14 [00:07<00:00,  1.94it/s]


...


Epoch 18/20
Train Loss: 0.1044
Val Loss:   0.5796
Val Acc:    81.11%
--------------------------------


Epoch 19/20: 100%|██████████| 14/14 [00:07<00:00,  1.91it/s]



Epoch 19/20
Train Loss: 0.0890
Val Loss:   0.6319
Val Acc:    80.00%
--------------------------------


Epoch 20/20: 100%|██████████| 14/14 [00:07<00:00,  1.94it/s]



Epoch 20/20
Train Loss: 0.1605
Val Loss:   0.6274
Val Acc:    82.22%
--------------------------------


In [ ]:
test_loss, test_acc = evaluate(test_loader)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.2f}%")

torch.save(model.state_dict(), "cleaning_cnn.pth")
print("Final model saved: cleaning_cnn.pth")


Test Loss: 0.6368
Test Accuracy: 80.00%
Final model saved: cleaning_cnn.pth


: 

In [ ]:

def predict_image(image_path, model, transform, device, classes):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    img_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(img_tensor)
        _, predicted = torch.max(outputs, 1)

    pred_class = classes[predicted.item()]
    print(f"Predicted Class: {pred_class}")
    return pred_class



In [31]:
classes = dataset.classes
predict_image(
    "dirty.webp",
    model,
    transform_val,
    device,
    classes
)

Predicted Class: high


'high'

Predicted Class: high


'high'